In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from galaxy_datasets.pytorch.galaxy_datamodule import CatalogDataModule
from galaxy_datasets.transforms import default_view_config, get_galaxy_transform\

from zoobot.pytorch.training.finetune import FinetuneableZoobotMetadataRegressor

/Users/user/mambaforge/envs/zoobot39_dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_train = pd.read_csv('imgs/train_dataset.csv')
df_test = pd.read_csv('imgs/test_dataset.csv')

In [4]:
df_train['X'] = df_train['X'].astype(np.float32)
df_train['y'] = df_train['y'].astype(np.float32)
df_test['X'] = df_test['X'].astype(np.float32)
df_test['y'] = df_test['y'].astype(np.float32)

In [5]:
df_train.head()

,image,label,id_str,file_loc,X,y
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0,a17e8ab7a7de9c79d3cf960af591bfd113669e405a9803...,/Users/user/repos/zoobot/examples/finetuning/t...,3.955509,9.051425
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1,2acac60ba3f744afb21f2e6c59d876949c8305e4f519cf...,/Users/user/repos/zoobot/examples/finetuning/t...,9.231839,23.476162
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0,50f422c99ac53ecdfb5559fb71e758c7052f3f4ad74e58...,/Users/user/repos/zoobot/examples/finetuning/t...,1.518943,2.276750
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,0,12fba589f6f7b0a3ce5c17039760b3c98e1cd6b69a9c1f...,/Users/user/repos/zoobot/examples/finetuning/t...,9.937435,25.216444
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1,f69e6578faa676168e96e9e4595fb79fff28aee64fd6bf...,/Users/user/repos/zoobot/examples/finetuning/t...,5.620438,16.640429


In [6]:
batch_size=16
num_workers=7


transform_cfg = default_view_config()
transform = get_galaxy_transform(transform_cfg)


datamodule = CatalogDataModule(
label_cols = ['y', 'X'],
catalog = df_train,
train_transform = transform,
test_transform = transform,
batch_size= batch_size,
num_workers= num_workers
)
datamodule.setup()

In [7]:


for batch in iter(datamodule.train_dataloader()):
    print(batch.keys())
    print(batch['image'].shape, batch['image'].dtype)
    print(batch['X'].shape, batch['X'].dtype)
    print(batch['y'].shape, batch['y'].dtype)
    break

/Users/user/mambaforge/envs/zoobot39_dev/lib/python3.9/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


dict_keys(['id_str', 'image', 'y', 'X'])
torch.Size([16, 3, 224, 224]) torch.float32
torch.Size([16]) torch.float32
torch.Size([16]) torch.float32


In [8]:
model = FinetuneableZoobotMetadataRegressor(
    # name='nassm/convnext_nano',
    name='hf_hub:mwalmsley/zoobot-encoder-convnext_pico',
    label_col='y',
    metadata_cols=['X']
)

In [ ]:
from zoobot.pytorch.training.finetune import get_trainer

datamodule.setup("fit")

save_dir = os.path.join('examples/finetuning/tmp/metadata/results')

trainer = get_trainer(save_dir, accelerator="auto", devices=1, strategy="auto", max_epochs=1, precision=32)
trainer.fit(model, datamodule)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/user/mambaforge/envs/zoobot39_dev/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name              | Type             | Params
-------------------------------------------------------
0 | encoder           | ConvNeXt         | 8.5 M 
1 | train_loss_metric | MeanMetric       | 0     
2 | val_loss_metric   | MeanMetric       | 0     
3 | test_loss_metric  | MeanMetric     